# Order and Delivery to Google Earth Engine

In this notebook we're going to cover how to order data using the [Orders API](https://developers.planet.com/apis/orders/) and deliver it to [Google Earth Engine (GEE)](https://earthengine.google.com/) using [Planet’s GEE Delivery integration](https://developers.planet.com/docs/integrations/gee/).

This example demonstrates how to:
1. Define the data to be ordered
2. Build a cloud delivery configuration object (`delivery_config`), which tells the Orders API where to deliver the data
3. Build an order request to be sent off to the Orders API, `iowa_order`
4. Create the order and have it deliver to your GEE project

**Prerequsites:** 
- An AOI - `iowa_aoi`
- Item ID(s) - `iowa_images`
- A GEE project with EE API enabled - `planet-devrel-dev`
- A pre-existing GEE [ImageCollection](https://developers.google.com/earth-engine/guides/ic_creating) - `gee-integration-testing`
- An account with a download quota. Not sure if you have download quota? Please [get in touch](https://www.planet.com/contact-sales/).

In [ ]:
import planet

## Define data to be ordered

In [ ]:
# The area of interest (AOI) defined as a polygon
iowa_aoi = {
    "type":
    "Polygon",
    "coordinates": [[[-91.198465, 42.893071], [-91.121931, 42.893071],
                     [-91.121931, 42.946205], [-91.198465, 42.946205],
                     [-91.198465, 42.893071]]]
}

# The item IDs we wish to order
iowa_images = ['20200925_161029_69_2223', '20200925_161027_48_2223']

## Define cloud delivery location
This is the *key concept* of this notebook. Here, we are defining a cloud delivery configuration object, `cloud_config`, where we are defining the destination to be Google Earth Engine, in the project named `planet-devrel-dev`, in the ImageCollection named `gee-integration-testing`.

In [ ]:
# Google Earth Engine configuration
cloud_config = planet.order_request.google_earth_engine(
    project='planet-devrel-dev', collection='gee-integration-testing')
# Order delivery configuration
delivery_config = planet.order_request.delivery(cloud_config=cloud_config)

## Build a simple order request
Here we are building an order request, where we specify the products we wish to order. Spesifically, here we are requesting the IDs defined in `iowa_images`, as `analytic_udm2` assets, from the Planet Scope Scene product, `PSScene`.

In [ ]:
# Product description for the order request
data_products = [
    planet.order_request.product(item_ids=iowa_images,
                                 product_bundle='analytic_udm2',
                                 item_type='PSScene')
]

# Build the order request
iowa_order = planet.order_request.build_request(name='iowa_order',
                                                products=data_products,
                                                delivery=delivery_config)

print(iowa_order)

## Create and deliver the order
Define a function that will create an order and update you with its progress. In this case, since we specified the delivery destination to be Google Earth Engine with `delivery_config`, it will also wait for it to be delivered to your project.

In [ ]:
async def create_and_deliver_order(order_request, client):
    '''Create and deliver an order.

    Parameters:
        order_request: An order request
        client: An Order client object
    '''
    with planet.reporting.StateBar(state='creating') as reporter:
        # Place an order to the Orders API
        order = await client.create_order(order_request)
        reporter.update(state='created', order_id=order['id'])
        # Wait while the order is being completed
        await client.wait(order['id'], callback=reporter.update_state)

    # Grab the details of the orders
    order_details = await client.get_order(order_id=order['id'])

    return order_details

Create and deliver the order `iowa_order` to the GEE project `planet-devrel-dev` in the ImageCollection `gee-integration-testing`.

In [ ]:
async with planet.Session() as ps:
    # The Orders API client
    client = planet.OrdersClient(ps)
    # Create the order and deliver it to GEE
    order_details = await create_and_deliver_order(iowa_order, client)

Display the results for the item data in the order.

In [ ]:
print(order_details['_links']['results'][0])

## Build an order request with clipping and harmonization applied
Here we are building an order request similar to the one above, but with [clipping](https://developers.planet.com/apis/orders/tools/#clip) and [harmonization](https://developers.planet.com/apis/orders/tools/#harmonization) applied. Namely, we are asking the Orders API to clip our images to the AOI, as defined by `iowa_aoi`, and radiometrically harmonizing the imagery with Sentinel-2's sensor.

In [ ]:
# Clip images to the AOI's perimeter and harmonize the data with Dove Classic
tools = [
    planet.order_request.harmonize_tool('Sentinel-2')
]

# Build the order request
iowa_order = planet.order_request.build_request(name='iowa_order',
                                                products=data_products,
                                                delivery=delivery_config,
                                                tools=tools)

print(iowa_order)
print(tools)

Create and deliver the clipped and harmonized data to `planet-devrel-dev` in `gee-integration-testing`.

In [ ]:
async with planet.Session() as ps:
    # The Orders API client
    client = planet.OrdersClient(ps)
    # Create the order and deliver it to GEE
    order_details = await create_and_deliver_order(iowa_order, client)

Display the results for the item data in the order.

In [ ]:
print(order_details['_links']['results'][0])